In [ ]:
#Importing all the requirements
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ta
#Reading the data
df=pd.read_csv('Binance-ETHUSDT-23-05-2020.csv')
df['timestamp']=pd.to_datetime(df['timestamp'],format='%Y-%m-%d %H:%M:%S')
df.set_index('timestamp',inplace=True)
df=df.astype(float)
#using TA
df1 = ta.add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume")
#Method for outcomes and scenarios 0s and 1s

In [ ]:
def scenarios(case,n=30,lower=-0.6,upper=1.2):
    case=case[['open','high','low','close','volume','trend_macd','trend_macd_signal','trend_macd_diff','momentum_rsi','volume_vwap']]
    Cumulated_name=[]
    Cumulated_boundary=[]
    for i in range(1,n+1):
        first_name='DiffClose'+str(i)
        second_name='DiffMin'+str(i)
        case[first_name]=case['close'].diff(i).shift(-i).fillna(0)/case['close']*100
        case[second_name]=np.where((case[first_name]<lower) | (case[first_name]>upper),i,n)
        Cumulated_name.append(first_name)
        Cumulated_boundary.append(second_name)
    case['Period']=case[Cumulated_boundary].min(axis=1)
    case['Outcome']=case.apply(lambda x:x['DiffClose'+str(x['Period']).split('.')[0]],axis=1)
    outcome=case[['open','high','low','close','volume','Period','Outcome','trend_macd','trend_macd_signal','trend_macd_diff','momentum_rsi','volume_vwap']]
    return outcome

In [ ]:
a=0
b=-0.3
matrix=[]
for i in range(13):
    a=a+0.3
    b=-0.3
    for j in range(13):
        matrix.append([np.round(a,1),np.round(b,1)])
        b=b-0.3
for i in matrix:
    final=scenarios(df1,n=200,upper=i[0],lower=i[1])
    final['Outcome']=np.where(final['Outcome']>0,str(i[0])+'%',str(i[1])+'%')
    final['Outcome']=np.where(final['Period']==200,'No Outcome',final['Outcome'])
    print(i)
    final.to_csv('Model_data'+str(i)+'.csv')

In [ ]:
indicator=pd.read_csv('Model_data[1.5, -1.5].csv')
sns.set(font_scale=1.3)
df=indicator.groupby('Outcome').size().reset_index()
df.columns=['Outcome','Cases %']
df['Total_Cases']=df['Cases %'].sum()
df['Cases %']=df['Cases %']/df['Total_Cases']*100
plt.figure(figsize=(14,7))
g=sns.barplot(data=df,x='Outcome',y='Cases %',palette='Pastel1')
plt.title('Combination Title')
for index, row in df.iterrows():
    g.text(
        row.name,
        row["Cases %"],
        str(round(row['Cases %'],2))+'%',
        fontsize=20,
        color="black",
        ha="center",
    )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib

for i in matrix:
    final=scenarios(case=df1,n=200,upper=i[0],lower=i[1])
    final['Outcome']=np.where(final['Outcome']>0,str(i[0])+'%',str(i[1])+'%')
    final['Outcome']=np.where(final['Period']==200,'No Outcome',final['Outcome'])
    
    final.to_csv('Model_data'+str(i)+'.csv')
    indicator=pd.read_csv('Model_data'+str(i)+'.csv')
    
    indicator["200MA"] = ta.trend.ema(indicator["close"], periods=200)
    indicator["uptrend"] = np.where(indicator["200MA"] < indicator["close"], 1, 0)
    indicator["trend_macd_diff1"] = indicator["trend_macd_diff"].shift(1)
    indicator['trend_macd_diff1']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(1)
    indicator['trend_macd_diff3']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(3)
    indicator['trend_macd_diff5']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(5)
    indicator['trend_macd_signal1']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(1)
    indicator['trend_macd_signal3']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(3)
    indicator['trend_macd_signal5']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(5)
    indicator['trend_macd1']=indicator['trend_macd']-indicator['trend_macd'].shift(1)
    indicator['trend_macd3']=indicator['trend_macd']-indicator['trend_macd'].shift(3)
    indicator['trend_macd5']=indicator['trend_macd']-indicator['trend_macd'].shift(5)
    indicator['trend']=(indicator['200MA']-indicator['close'])/indicator['close']
    indicator['volume_vwap']=indicator['volume_vwap']/indicator['close']

    Initial=indicator[['trend_macd','trend_macd_signal','trend_macd_diff','trend_macd_diff1','trend_macd_diff3','trend_macd_diff5','trend_macd_signal1','trend_macd_signal3','trend_macd_signal5','trend_macd1','trend_macd3','trend_macd5','trend','Outcome','momentum_rsi', 'volume_vwap']].dropna()
    Initial=Initial[Initial['Outcome']!='No Outcome']
    Initial['Outcome']=Initial['Outcome'].apply(lambda x:len(x))
    Initial['Outcome']=np.where(Initial['Outcome']>4,0,1)

    from sklearn.model_selection import StratifiedShuffleSplit

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

    for train_index, test_index in sss.split(Initial.drop('Outcome',axis=1), Initial['Outcome']):
        X_train, X_test = Initial.drop('Outcome',axis=1).iloc[train_index], Initial.drop('Outcome',axis=1).iloc[test_index]
        y_train, y_test = Initial['Outcome'].iloc[train_index], Initial['Outcome'].iloc[test_index]

    Classifier = RandomForestClassifier(max_depth=11,min_samples_leaf=400,max_features=11,min_samples_split=20,n_estimators=1000, random_state=28, n_jobs=-1)
    Classifier.fit(X_train, y_train)
    y_pred = Classifier.predict(X_test)

    joblib.dump(Classifier, 'Classifier'+str(i)+'.pkl')

In [ ]:
from joblib import load
for i in matrix:
    Classifier_values=i
    Classifier = load('Classifier'+str(i)+'.pkl')  
    indicator=pd.read_csv('Model_data'+str(i)+'.csv')

    indicator["200MA"] = ta.trend.ema(indicator["close"], periods=200)
    indicator["uptrend"] = np.where(indicator["200MA"] < indicator["close"], 1, 0)
    indicator["trend_macd_diff1"] = indicator["trend_macd_diff"].shift(1)
    indicator['trend_macd_diff1']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(1)
    indicator['trend_macd_diff3']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(3)
    indicator['trend_macd_diff5']=indicator['trend_macd_diff']-indicator['trend_macd_diff'].shift(5)
    indicator['trend_macd_signal1']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(1)
    indicator['trend_macd_signal3']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(3)
    indicator['trend_macd_signal5']=indicator['trend_macd_signal']-indicator['trend_macd_signal'].shift(5)
    indicator['trend_macd1']=indicator['trend_macd']-indicator['trend_macd'].shift(1)
    indicator['trend_macd3']=indicator['trend_macd']-indicator['trend_macd'].shift(3)
    indicator['trend_macd5']=indicator['trend_macd']-indicator['trend_macd'].shift(5)
    indicator['trend']=(indicator['200MA']-indicator['close'])/indicator['close']
    indicator['volume_vwap']=indicator['volume_vwap']/indicator['close']

    Initial=indicator[['trend_macd','trend_macd_signal','trend_macd_diff','trend_macd_diff1','trend_macd_diff3','trend_macd_diff5','trend_macd_signal1','trend_macd_signal3','trend_macd_signal5','trend_macd1','trend_macd3','trend_macd5','trend','Outcome','momentum_rsi', 'volume_vwap']].dropna()
    no_outcome=len(Initial[Initial['Outcome']=='No Outcome'])/len(Initial[Initial['Outcome']!='No Outcome'])
    Initial=Initial[Initial['Outcome']!='No Outcome']
    Initial['Outcome']=Initial['Outcome'].apply(lambda x:len(x))
    Initial['Outcome']=np.where(Initial['Outcome']>4,0,1)
    print(Initial.info())
    from sklearn.model_selection import StratifiedShuffleSplit

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

    for train_index, test_index in sss.split(Initial.drop('Outcome',axis=1), Initial['Outcome']):
        X_train, X_test = Initial.drop('Outcome',axis=1).iloc[train_index], Initial.drop('Outcome',axis=1).iloc[test_index]
        y_train, y_test = Initial['Outcome'].iloc[train_index], Initial['Outcome'].iloc[test_index]
    

    predictions_proba = Classifier.predict_proba(X_test)
    predictions_probability = pd.concat(
    [pd.DataFrame(predictions_proba), y_test.reset_index()['Outcome']], ignore_index=True,axis=1
)
    predictions_probability.columns = ["prob_0", "prob_1", "result"]

    predictions = Classifier.predict(X_test)
    predictions_1 = pd.concat([pd.DataFrame(predictions), pd.DataFrame(y_test)], axis=1)
    predictions_1.columns = ["prediction", "result"]
    Results = pd.DataFrame([], columns=["P(p)>", "Success rate", "Quantity"])
    for j in range(1, 100):
        P_p = np.round(0 + j / 100, 3)

        Quantity = len(
        (
            predictions_probability[
                (predictions_probability["prob_1"] >= (P_p))
            ]
        )
    )
        if Quantity == 0:
            break
        Success_rate = np.round(
        (
            predictions_probability[
                (predictions_probability["prob_1"] >= (P_p))
            ]["result"]
        ).sum()
        / len(
            (
                predictions_probability[
                    (predictions_probability["prob_1"] >= (P_p))
                ]
            )
        ),
        3,
    )
        Results = Results.append(
        {"P(p)>": P_p, "Success rate": Success_rate, "Quantity": Quantity},
        ignore_index=True,
    )
    Results['no_outcome']=no_outcome
    Results.to_csv('Results'+str(i)+'.csv')

In [ ]:
Results=pd.DataFrame([])
sns.set(font_scale=1.2)
for i in matrix:
    results1=pd.read_csv('Results'+str(i)+'.csv')
    Positive_Fees=(i[0]+100)*0.00075+100*0.00075
    Negative_Fees=(i[1]+100)*0.00075+100*0.00075
    No_Outcome_Fees=200*0.00075
    results1['Return']=(i[0]-Positive_Fees)*results1['Success rate']*results1['Quantity']+(i[1]-Positive_Fees)*(results1['Quantity']-results1['Success rate']*results1['Quantity'])+((i[0]+i[1])/7-No_Outcome_Fees)*results1['Quantity']*results1['no_outcome']
    results1['Combination']=str(i[0])+', '+str(i[1])
    results1.drop(results1.columns[0],inplace=True,axis=1)
    Results=Results.append(results1,ignore_index=True)
selected_lines=Results.sort_values(by='Return',ascending=False)['Combination'].unique()[0:5]
from celluloid import Camera
camera = Camera(plt.figure(figsize=(11,7)))
plt.title('Models Return')
plt.xlabel('Models Return')
plt.ylabel('Models Return')

for i in range(75):
    print(i)
    line=Results[(Results['Combination'].isin(selected_lines)) & (Results['P(p)>']<0.16+i/100)]
    if(i==1):
        sns.lineplot(data=line,x='P(p)>',y='Return',hue='Combination')
    else:
        sns.lineplot(data=line,x='P(p)>',y='Return',hue='Combination',legend=False,lw=0.5)
        sns.scatterplot(data=line,x='P(p)>',y='Return',hue='Combination',legend=False)
        #plt.plot('P(p)>','Return',data=line[line['Combination']==j],lw=0.5)
        

    camera.snap()
print('a')
anim = camera.animate(blit=True,interval=100)
print('a')
anim.save('scatter'+str(i)+'.gif')


In [ ]:
Results=Results[(Results['Combination']=='1.5, -1.5') & (Results['P(p)>']>0.47) & (Results['P(p)>']<0.72)]
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

sns.set(font_scale=1.1)
sns.set_style('dark')

pastel=cm.get_cmap('Pastel1',len(Results))
diverging=cm.get_cmap('RdYlGn',len(Results))
plt.figure(figsize=(14,8))
plt.bar(x=Results['P(p)>'], height=Results['Success rate'],width=0.005,color= pastel(np.linspace(0,1,len(Results))))
plt.ylabel('Success Rate')  
plt.xlabel('P(p)>')
for index, row in Results.iterrows():
    plt.text(
        row['P(p)>'],
        row["Success rate"],
        int(row.Quantity),
        fontsize=12,
        color="black",
        ha="center",
    )
axes2 = plt.twinx()
axes2.scatter(Results['P(p)>'], Results['Return'], color=diverging(scaler.fit_transform(np.array(Results['Return']).reshape(-1,1)).reshape(len(Results),1)).reshape(len(Results),4),s=200)
axes2.plot(Results['P(p)>'], Results['Return'],lw=0.3,color='grey')
axes2.set_ylabel('Return',color='red')
plt.xticks(Results['P(p)>'])